In [2]:
import os, requests
from dotenv import load_dotenv
load_dotenv()

#RIOT KEY is generated by Riot web page : 
API_KEY = os.getenv("RIOT_API_KEY")
PLATFORM = "kr"    # league-v4, summoner-v4
REGIONAL = "asia"  # match-v5
HEAD = {"X-Riot-Token": API_KEY}

print("API_KEY set:", API_KEY is not None, " (prefix: ", (API_KEY or "")[:8], ")")
print("PLATFORM base:", f"https://{PLATFORM}.api.riotgames.com")
print("REGIONAL base:", f"https://{REGIONAL}.api.riotgames.com")


API_KEY set: True  (prefix:  RGAPI-97 )
PLATFORM base: https://kr.api.riotgames.com
REGIONAL base: https://asia.api.riotgames.com


In [3]:
url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"
r = requests.get(url, headers=HEAD, timeout=15)
print("status:", r.status_code)
data = r.json()
print("top-level keys:", list(data.keys()))

entries = data.get("entries", [])
print("entries count:", len(entries))
if entries:
    print("sample entry keys:", list(entries[0].keys()))
    print("sample entry snippet:", {k: entries[0].get(k) for k in ["summonerId","puuid","summonerName","leaguePoints"]})


status: 200
top-level keys: ['tier', 'leagueId', 'queue', 'name', 'entries']
entries count: 300
sample entry keys: ['puuid', 'leaguePoints', 'rank', 'wins', 'losses', 'veteran', 'inactive', 'freshBlood', 'hotStreak']
sample entry snippet: {'summonerId': None, 'puuid': 'vILWZj1-w6mLKBQtW7R0_mYwz_fbFEWVwOkNHW3cvR23WZ2YKYfnifGwmfQD4itRXiJknhs3Rqn0xg', 'summonerName': None, 'leaguePoints': 2017}


In [ ]:
candidates = [e for e in entries if "puuid" in e][:5]
print("candidate count:", len(candidates))
for i, e in enumerate(candidates, 1):
    print(f"{i}) puuid prefix:", e["puuid"][:18], "  name:", e.get("summonerName"))

test_puuid = candidates[0]["puuid"]
url = f"https://{PLATFORM}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{test_puuid}"
r = requests.get(url, headers=HEAD, timeout=15)
print("GET summoner/by-puuid status:", r.status_code)
try:
    summ = r.json()
    print("summ keys:", list(summ.keys()))
    print("status node?:", isinstance(summ, dict) and "status" in summ)
    print("id present?:", isinstance(summ, dict) and ("id" in summ))
    print("name:", summ.get("name"), " level:", summ.get("summonerLevel"))
except Exception as e:
    print("JSON parse error:", e, " raw text:", r.text[:200])


candidate count: 5
1) puuid prefix: vILWZj1-w6mLKBQtW7   name: None
2) puuid prefix: hpY6W3i6PBrdICOBoH   name: None
3) puuid prefix: pUgQ0wkEJpQUFRtfil   name: None
4) puuid prefix: qiL-YkEC_RO9Tuzflz   name: None
5) puuid prefix: amH7Caz9jLiqRkTmqq   name: None
GET summoner/by-puuid status: 200
summ keys: ['puuid', 'profileIconId', 'revisionDate', 'summonerLevel']
status node?: False
id present?: False
name: None  level: 764


In [5]:
REGIONAL = "asia"
HEAD = {"X-Riot-Token": API_KEY}

test_puuid = candidates[0]["puuid"]
url = f"https://{REGIONAL}.api.riotgames.com/riot/account/v1/accounts/by-puuid/{test_puuid}"
r = requests.get(url, headers=HEAD, timeout=15)
print("account-v1 status:", r.status_code)
acc = r.json()
print("Riot ID:", acc.get("gameName"), "#", acc.get("tagLine"))


account-v1 status: 200
Riot ID: DRX kyeahoo # 0813


In [6]:
url = f"https://{REGIONAL}.api.riotgames.com/lol/match/v5/matches/by-puuid/{test_puuid}/ids"
r = requests.get(url, headers=HEAD, params={"queue":420, "start":0, "count":5}, timeout=15)
print("match-ids status:", r.status_code)
print(r.json())


match-ids status: 200
['KR_7880912321', 'KR_7880747281', 'KR_7880664793', 'KR_7880600705', 'KR_7880547537']


In [7]:
test_mid_list = ['KR_7866633673', 'KR_7866531451', 'KR_7866485063'] 

for mid in test_mid_list:
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{mid}"
    rr = requests.get(url, headers=HEAD, timeout=15)
    print("match", mid, "status:", rr.status_code)
    m = rr.json()
    info = m.get("info", {})
    gv = info.get("gameVersion")
    parts = info.get("participants", [])
    me = next((p for p in parts if p.get("puuid")==test_puuid), None)
    print("  gameVersion:", gv, "  found_me:", me is not None)


match KR_7866633673 status: 200
  gameVersion: 15.20.717.2831   found_me: True
match KR_7866531451 status: 200
  gameVersion: 15.20.717.2831   found_me: True
match KR_7866485063 status: 200
  gameVersion: 15.20.717.2831   found_me: True


In [8]:
# 방금 5개 id에서 15.20 매치만 추리기
kept = []
for mid in ['KR_7866633673', 'KR_7866531451', 'KR_7866485063', 'KR_7866387946', 'KR_7864579918']:
    m = requests.get(f"https://asia.api.riotgames.com/lol/match/v5/matches/{mid}", headers=HEAD, timeout=15).json()
    gv = str(m.get("info", {}).get("gameVersion", ""))
    if gv.startswith("15.20"):
        kept.append((mid, gv))
print("kept (15.20.x):", kept)


kept (15.20.x): [('KR_7866633673', '15.20.717.2831'), ('KR_7866531451', '15.20.717.2831'), ('KR_7866485063', '15.20.717.2831'), ('KR_7866387946', '15.20.717.2831'), ('KR_7864579918', '15.20.717.2831')]


In [9]:
for mid, gv in kept[:2]:
    tl = requests.get(f"https://asia.api.riotgames.com/lol/match/v5/matches/{mid}/timeline", headers=HEAD, timeout=15)
    print("timeline", mid, "status:", tl.status_code, "len:", len(tl.text))


timeline KR_7866633673 status: 200 len: 398360
timeline KR_7866531451 status: 200 len: 596304
